# Using MFA to determine the inventory

## Finding out how much of a certain resource comes from the various inputs

In [2]:
import bw2data as bd
import bw2calc as bc

14:40:35+0000 [warning  ] Can't import `SimaProBlockCSVImporter` - please install `bw2io` with `pip install bw2io[multifunctional]` or install `multifunctional` and `bw_simapro_csv` manually.


In [3]:
bd.projects.set_current("ecoinvent-3.12-cutoff")

Look at PV production

In [18]:
laminate = bd.get_node(name="photovoltaic laminate production, CIS", location="RoW", database="ecoinvent-3.12-cutoff")

Find a resource - in this case silver

In [19]:
[
    x 
    for x in bd.Database("ecoinvent-3.12-biosphere") 
    if "silver" in x['name'].lower() 
    and x['unit'] == 'kilogram'
    and x['categories'][0] == 'natural resource'
]

['Silver' (kilogram, None, ('natural resource', 'in ground'))]

In [20]:
silver = bd.get_node(name="Silver", categories=('natural resource', 'in ground'), database="ecoinvent-3.12-biosphere")

Set up the LCA to calculate the life cycle inventory

In [21]:
fu, data_objs, _ = bd.prepare_lca_inputs(demand={laminate: 1})

In [22]:
lca = bc.LCA(fu)
lca.lci()

Get total amount of silver

In [23]:
lca.inventory[lca.dicts.biosphere[silver.id], :].sum()

0.00010398154729303756

Look at the inputs, and group by reference product.

In [26]:
from collections import defaultdict
from tqdm import tqdm

results = defaultdict(float)

for edge in tqdm(laminate.technosphere()):
    lca.lci(demand={edge.input.id: edge['amount']})
    results[edge.input['reference product']] += lca.inventory[lca.dicts.biosphere[silver.id], :].sum()

100%|██████████| 131/131 [00:08<00:00, 15.24it/s]


In [27]:
results = sorted([(amount, product) for product, amount in results.items()], reverse=True)
results

[(3.75187753790523e-05, 'indium'),
 (1.6436978459997453e-05, 'selenium'),
 (9.507742016907456e-06, 'molybdenum'),
 (9.328407043433794e-06, 'cadmium sulfide, semiconductor-grade'),
 (8.502707197857379e-06, 'copper, cathode'),
 (7.521691257057335e-06, 'solar glass, low-iron'),
 (4.772340624321925e-06, 'photovoltaic panel factory'),
 (4.202992320313527e-06, 'zinc'),
 (3.035701464425767e-06, 'electricity, medium voltage'),
 (1.1730615622690166e-06, 'ethylvinylacetate, foil'),
 (8.653200500744005e-07, 'tempering, flat glass'),
 (7.010044970381689e-07, 'gallium, high-grade'),
 (1.8501945393076774e-07, 'urea'),
 (1.3398674029736132e-07, 'ammonia, anhydrous, liquid'),
 (4.9018909602120654e-08, 'nitrogen, liquid'),
 (2.4064055181474113e-08, 'tin'),
 (1.2205880817380435e-08, 'acetone, liquid'),
 (5.875025812187578e-09, 'waste plastic, mixture'),
 (2.9325065817419356e-09, 'waste, from silicon wafer production, inorganic'),
 (8.219555116635906e-10, 'tap water'),
 (7.862548160409611e-10, 'argon, li

## Injecting MFA results into LCA calculations

If we want to take the MFA results as the single source of truth for our calculations, we can inject these results into our inventory before doing impact assessment.

If we want to set this as the total amount, then we need to first remove all other processes which use that resource or emit that emission:

In [28]:
from scipy import sparse

In [32]:
def zero_amount_emission_in_inventory(lca: bc.LCA, flow: bd.Node) -> None:
    """Set all values for elementary flow `flow` to zero in `lca.inventory` matrix"""
    ident_matrix = sparse.identity(lca.inventory.shape[0], format="csr")
    index = lca.dicts.biosphere[flow.id]
    ident_matrix[index, index] = 0
    lca.inventory = ident_matrix @ lca.inventory
    assert not lca.inventory[index, :].sum()

In [33]:
zero_amount_emission_in_inventory(lca, silver)

Now we can set the amount manually.

In [35]:
lca.inventory[
    lca.dicts.biosphere[silver.id],
    lca.dicts.activity[laminate.id]
] = 12345.6

/opt/tljh/user/lib/python3.12/site-packages/scipy/sparse/_index.py:108: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil and dok are more efficient.
  self._set_intXint(row, col, x.flat[0])


In [36]:
lca.inventory[lca.dicts.biosphere[silver.id]].sum()

12345.6

You can now do LCIA as normal.